# Glider Speed polars analysis notebook

In [95]:
import numpy as np
import math
from numpy.polynomial import Polynomial

print(np.__version__)

# https://www.geeksforgeeks.org/python-implementation-of-polynomial-regression/?ref=lbp
# old vs new API https://numpy.org/doc/1.21/reference/routines.polynomials.html

1.21.1


In [96]:
FILE_TO_LOAD = './lak19-polars-db.json'
#FILE_TO_LOAD = './ventus2-15m-polars-db.json'


In [97]:
# utilities functions & constants

KM_TO_MS = 3.6				# factor to convert km/h in m/s
CONVERT_TO_MS = False		# True if we want to convert speed from km/h to m/s

POLAR_CURVE_START_KM = 70		# in km/h
POLAR_CURVE_END_KM = 230		# in km/h

POLAR_CURVE_START = (POLAR_CURVE_START_KM / KM_TO_MS) if CONVERT_TO_MS else POLAR_CURVE_START_KM
POLAR_CURVE_END = (POLAR_CURVE_END_KM / KM_TO_MS)if CONVERT_TO_MS else POLAR_CURVE_END_KM

POLAR_CURVE_NBR_SAMPLE =  33

MASS_UNITS = { 'wing_loading': 'kg/m2', 'weight' : 'kg'}		# Some units use for polar curves relative to the glider mass
MASS_TXT = { 'wing_loading': 'wing loading', 'weight' : 'weight'}

def polar_to_string(p):
	polar_str = '{}x<sup>2</sup>'.format(round(p.coeffs[0],4))
	polar_str += (' + ' if (p.coeffs[1] >= 0 ) else ' ')
	polar_str += '{}x'.format(round(p.coeffs[1],4))
	polar_str += (' + ' if (p.coeffs[2] >= 0 ) else ' ')
	polar_str += '{}'.format(round(p.coeffs[2],4))
	return polar_str

# to convert speed from km/h to m/s
xaxis_unit = lambda x: x/ (KM_TO_MS if CONVERT_TO_MS else 1)

In [98]:
# To plot  polar when x&y are 'relevé' by hand
def data4plotly_by_hand(item, mass_key):
	polar_data = dict()

	wl = item[mass_key]
	tn = 'By hand ({}: {} {})'.format(MASS_TXT[mass_key], wl, MASS_UNITS[mass_key])

	polar_data["x"] = item['speed']
	polar_data["y"] = item['sink_rate']
	polar_data["L/D"] = None					# TODO
	polar_data["serie_name"] = tn
	polar_data["polynomial"] = None
	return polar_data

In [99]:
# To plot a polar curve when it is defined by 3 coefficients, y = - (Ax^2 + Bx + C)
def data4plotly_ABC(item, mass_key):
	polar_data = dict()

	wl = item[mass_key]
	tn = 'Model ABC ({}: {} {})'.format(MASS_TXT[mass_key], wl, MASS_UNITS[mass_key])

	polar = np.poly1d([item["A"],item["B"], item["C"] ])
	polar = np.polymul(polar,-1)
	print ('\nModel ABC, polynomial ({}: {} {}) is'.format(MASS_TXT[mass_key], wl, MASS_UNITS[mass_key]))
	print(polar) 

	x_polar = np.linspace(POLAR_CURVE_START, POLAR_CURVE_END, POLAR_CURVE_NBR_SAMPLE)
	# print('\r\nVitesse (km/h)= {}'.format(x_polar))

	y_polar = polar(np.divide(x_polar,100))
	# print('\r\nSink rate (m/s) = {}'.format(y_polar))

	ld = np.divide(np.divide(-x_polar , KM_TO_MS), y_polar)
	# print('\r\nL/D = {}'.format(ld))

	polar_data["x"] = x_polar
	polar_data["y"] = y_polar
	polar_data["L/D"] = ld
	polar_data["serie_name"] = tn
	polar_data["polynomial"] = polar
	return polar_data

In [100]:
# To plot polar curve when it is defined by 3 points (Vi in km/h, Vz in m/s)
def data4plotly_3_points(item, mass_key):
	polar_data = dict()

	speeds = [xaxis_unit(x) for x in item['speed']]
	sink_rate = item['sink_rate']
	wl = item[mass_key]
	tn = 'Model 3-points ({}: {} {})'.format(MASS_TXT[mass_key], wl, MASS_UNITS[mass_key])

	# Fit a polynomial of degree 2 based on the 3 points 
	polar = np.poly1d(np.polyfit(speeds, sink_rate, 2))
	print ('\nModel 3 points, polynomial ({}: {} {}) is'.format(MASS_TXT[mass_key], wl, MASS_UNITS[mass_key]))
	print(polar) 

	# The curve will be the calculated polar using this polynomial, 	
	x_polar = np.linspace(POLAR_CURVE_START, POLAR_CURVE_END, POLAR_CURVE_NBR_SAMPLE)
	y_polar = polar(x_polar)
	# print('\r\nc3_y_polar = {}'.format(y_polar))

	polar_data["x"] = x_polar
	polar_data["y"] = y_polar
	polar_data["L/D"] = None					# TODO
	polar_data["serie_name"] = tn
	polar_data["polynomial"] = polar
	return polar_data

In [101]:

## Polar data import from json file
import json
glider_pdata = None	# glider polars data

def data_for_plotly(item, mass_key):
	if item['method'] == 'by-hand':
		return data4plotly_by_hand(item, mass_key)
	elif item['method'] == 'ABC':
		return data4plotly_ABC(item, mass_key)
	elif item['method'] == '3-points':
		return data4plotly_3_points(item, mass_key)
	else:
		print('Error, method {} is unknow.'.format(item['method']))

# Opening JSON file
with open(FILE_TO_LOAD) as json_file:
	glider_pdata = json.load(json_file)

# Wing loading as unit used for a given polar curve else assume it's the weight
WL_AS_REF_UNIT = 'wing_loading' if (('gp_mass_unit' in glider_pdata.keys()) and (glider_pdata['gp_mass_unit'] == 'wing_loading')) else 'weight'

# Then compute the different curves to plot
polars_ref_data = []
polars_mdl_data = None

print(glider_pdata['specifications'].keys())

for key, value in glider_pdata['specifications'].items():
	if key.startswith('reference'):
		value['polar'] = data_for_plotly(value, WL_AS_REF_UNIT)
		polars_ref_data.append(value)
	elif key.startswith('model'):
		value['polar'] = data_for_plotly(value, WL_AS_REF_UNIT)
		polars_mdl_data = value
	else:
		print('Error, specification {} is unknow !'.format(key))

# print (polars_ref_data)
# print (polars_mdl_data)


dict_keys(['reference-1', 'reference-2', 'reference-3', 'model'])

Model 3 points, polynomial (wing loading: 31.5 kg/m2) is
            2
-0.0001267 x + 0.01687 x - 1.02

Model ABC, polynomial (wing loading: 31.6 kg/m2) is
       2
-1.24 x + 1.61 x - 0.97

Model 3 points, polynomial (wing loading: 30.1 kg/m2) is
            2
-0.0002022 x + 0.03674 x - 2.133

Model 3 points, polynomial (wing loading: 35.7 kg/m2) is
            2
-0.0002167 x + 0.04017 x - 2.45


In [102]:

# calculate polar for a specific weight (or wing loading) 
def polarAdjusted4Weight(new_weight, x, y, old_weight):
	new_x = [ vi * math.sqrt(new_weight/old_weight) for vi in x]
	new_y = [ vz * math.sqrt(new_weight/old_weight) for vz in y]

	return new_x, new_y

In [103]:
# Display the graph

# https://plotly.com/python/plotly-fundamentals/
# https://plotly.com/python-api-reference/
# https://ipywidgets.readthedocs.io/en/stable/

import plotly.graph_objects as go
from ipywidgets import widgets, Layout, Label
import ipywidgets as iw

traces=[]

# add reference polar curves
for aPolar in polars_ref_data:
	if aPolar['polar'] is not None:
		trace = go.Scatter( x=aPolar['polar']['x'], y=aPolar['polar']['y'], mode='lines', 
			name = '<b>{}</b><br><i>{}</i><br>'.format(aPolar['name'],aPolar['polar']['serie_name']),
			hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y', line = dict(dash='dash'))
		traces.append(trace)

# add traces for the model curves, the one we want to modify parameters (weight or wing loading) and interpolate new polar curve
if polars_mdl_data is not None:
	traces.append(go.Scatter( x=polars_mdl_data['polar']['x'], y=polars_mdl_data['polar']['y'], mode='lines', 
		name = '<b>{}</b><br><i>{}</i>'.format(polars_mdl_data['name'],polars_mdl_data['polar']['serie_name']),
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y', ))
	if WL_AS_REF_UNIT == 'wing_loading':
		traces.append(go.Scatter( x=polars_mdl_data['speed'], y=polars_mdl_data['sink_rate'], mode='markers', name='p1, p2, p3 points', marker_symbol = 'diamond',marker_size=10,
			text=['Point #1', 'Point #2', 'Point #3'], hovertemplate='<extra></extra><b>%{text}</b><br>Speed: %{x}km/h<br>Sink rate: %{y}m/s', hoverinfo='x+y+text'))
	traces.append(go.Scatter( x=polars_mdl_data['polar']['x'], y=polars_mdl_data['polar']['y'], mode='lines+markers', 
		name = 'Adjusted ({}: {} {})'.format(MASS_TXT[WL_AS_REF_UNIT], polars_mdl_data[WL_AS_REF_UNIT], MASS_UNITS[WL_AS_REF_UNIT]),
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y', ))

layout = go.Layout(
		title_text='<b>{} - Speed polars analysis</b><br><span class="font-size: smaller;">(wing area: {}m2)</span>'.format(glider_pdata['glider'], glider_pdata['wing_area']),
		height=800,
		yaxis=dict(range=[-4, 1]),
		template='ggplot2',
	)

fig = go.Figure(data=traces, layout=layout)

fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black',  dtick=0.5)
fig.update_yaxes(title_text='Vz (m/s)', title_font=dict(size=14, family='Courier', color='crimson'))

fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black')
fig.update_xaxes(title_text='Vitesse ({})'.format(( 'm/s' if CONVERT_TO_MS else 'km/h')),
	 title_font=dict(size=14, family='Courier', color='crimson'))

if polars_mdl_data is not None:
	fig.add_annotation(x=xaxis_unit(50), y=-2.75, text='Polar: {}'.format( polar_to_string(polars_mdl_data['polar']['polynomial'])), showarrow=False)

fig.update_layout(xaxis_range=[0,max(traces[0].x)*1.2])

# use FigureWiget insead of fig.show() to add controls
# fig.show()
f2 = go.FigureWidget(fig)

# if wing-loading
if WL_AS_REF_UNIT == 'wing_loading':
	ref_init_alue = polars_mdl_data['wing_loading'] 
	ref_unit = MASS_UNITS['wing_loading']
	ref_desc = MASS_TXT['wing_loading']

	weights = widgets.FloatSlider(
		value=polars_mdl_data['wing_loading'],
		min=29,
		max=52,
		step=0.1,
		description= MASS_TXT['wing_loading'],
		disabled=False,
		continuous_update=True,
		# orientation='vertical',
		readout=False,
		readout_format='.1f',
	)
	weight_text = Label(value=f'{weights.value:.1f} '+ref_unit, description='', layout=Layout(width='80px'))
else: 
	ref_init_alue = polars_mdl_data['weight'] 
	ref_unit = MASS_UNITS['weight']
	ref_desc = MASS_TXT['weight']

	weights = widgets.IntSlider(
		value=ref_init_alue,
		min=300,
		max=550,
		step=1,
		description=ref_desc,
		continuous_update=True,
		readout=False 
	)
	weight_text = Label(value=f'{weights.value:d} '+ref_unit, description='', layout=Layout(width='80px'))

container = widgets.HBox(children=[weights,weight_text,], layout=Layout(margin='0px 0px 100px 100px'))

def on_value_change(change):
	if WL_AS_REF_UNIT == 'wing_loading':
		weight_text.value = f'{weights.value:.1f} '+ref_unit
	else:
		weight_text.value = f'{weights.value:d} '+ref_unit
	with f2.batch_update():
		x, y = polarAdjusted4Weight(weights.value, polars_mdl_data['polar']['x'], polars_mdl_data['polar']['y'],ref_init_alue)
		f2.data[-1].x = x
		f2.data[-1].y = y
		
		t = 'Adjusted ({}: {} {})'.format(ref_desc, weights.value, ref_unit )
		f2.data[-1].name = t

weights.observe(on_value_change, names="value")
widgets.VBox([f2, container])


    'data': [{'hoverinfo': 'x+y',
              'hovertemplate': '<extra></extra…